<a href="https://colab.research.google.com/github/dadakys/cnn-classification-xray-images/blob/main/xray_classification_nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# Install dependencies
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from pathlib import Path
from google.colab import drive
import shutil

# 📁 Mount Google Drive
drive.mount('/content/drive')

# 🧭 Set the path to your dataset on Google Drive
drive_data_dir = "/content/drive/MyDrive/SXOLH2/ex8/Neural_Networks/COVID-19_Radiography_Dataset"

# 📥 Copy dataset to Colab's local storage (faster access)
local_data_dir = "/content/COVID-19_Radiography_Dataset"

if not os.path.exists(local_data_dir):  # avoid copying multiple times
    print("📥 Copying dataset to Colab local storage...")
    shutil.copytree(drive_data_dir, local_data_dir)
else:
    print("✅ Dataset already in local storage.")

# 📁 Now use local_data_dir for loading images (not Google Drive anymore)
data_dir = local_data_dir


In [ ]:
#check the hardware used
import tensorflow as tf
print("Tensorflow version:", tf.__version__)
print("GPU available:", tf.config.list_physical_devices('GPU'))


In [ ]:

# Create  dataframe excluding masks (collect dataset)
filepaths = []
labels = []

for label in os.listdir(data_dir):
    label_path = os.path.join(data_dir, label)
    if not os.path.isdir(label_path):
        continue

    for subfolder in os.listdir(label_path):
        sub_path = os.path.join(label_path, subfolder)
        if subfolder.lower() == 'masks':
            continue

        if os.path.isdir(sub_path):
            for file in os.listdir(sub_path):
                if file.endswith('.png'):
                    filepaths.append(os.path.join(sub_path, file))
                    labels.append(label)
        elif subfolder.endswith('.png'):
            filepaths.append(sub_path)
            labels.append(label)

df = pd.DataFrame({'filepaths': filepaths, 'labels': labels})

print(f"✅ Collected {len(df)} images!")
print(df['labels'].value_counts())
df.head()


In [ ]:
df.info()

In [ ]:
#plot images per class
import matplotlib.pyplot as plt
import cv2

# 📊 Get one sample per unique class
sample_df = df.groupby("labels").apply(lambda x: x.sample(1, random_state=42)).reset_index(drop=True)

# 🎨 Plot the images
plt.figure(figsize=(12, 6))

for i, row in enumerate(sample_df.itertuples()):
    img = cv2.imread(row.filepaths)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    plt.subplot(1, 4, i + 1)
    plt.imshow(img)
    plt.title(row.labels)
    plt.axis("off")

plt.suptitle("Images from Each Class")
plt.tight_layout()
plt.show()


In [ ]:
#images class distribution
#VISUALIZATIONS
# Churn distribution with value labels
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(6, 5))
ax = sns.countplot(data=df, x='labels')
plt.title('Dataset Class Distribution')
plt.xlabel('Classes')
plt.ylabel('Images')

# Add count labels on top of each bar
for p in ax.patches:
    height = p.get_height()
    ax.text(p.get_x() + p.get_width() / 2., height + 100, int(height), ha="center", fontsize=12)

plt.show()


In [ ]:
  # Split the dataset with stratified folds
  from sklearn.model_selection import StratifiedKFold

  K = 4
  skf = StratifiedKFold(n_splits=K, shuffle=True, random_state=42)

  folds = []

  for train_idx, test_idx in skf.split(df['filepaths'], df['labels']):
      train_data = df.iloc[train_idx].reset_index(drop=True)
      test_data = df.iloc[test_idx].reset_index(drop=True)
      folds.append((train_data, test_data))


In [ ]:
#Add data augmentation and normalize images
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = (224, 224)
BATCH_SIZE = 256

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)


In [ ]:
from tensorflow.keras import mixed_precision

# Enable mixed precision
mixed_precision.set_global_policy('mixed_float16')


In [ ]:
#UPDATED
#CNN from scratch
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model

# ➡️ Συνάρτηση για να δημιουργούμε νέο "καθαρό" μοντέλο κάθε φορά
def create_model():
    model = Sequential()

    # 🔹 Conv Layer 1
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # 🔹 Conv Layer 2
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # 🔹 Conv Layer 3
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # 🔹 Flatten και Fully Connected
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))

    # 🔹 Output Layer
    model.add(Dense(4, activation='softmax', dtype='float32'))

    # ✅ Compile
    model.compile(
        optimizer=Adam(learning_rate=0.0005),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    # 📊 Summary
    model.summary()

    # 🖼️ Plot architecture to file
    plot_model(model, to_file='model_architecture.png', show_shapes=True, show_layer_names=True)

    return model


In [ ]:
#MobileNetV2
# ================================================================
# Transfer-learning baseline: MobileNet V2 on 4-fold CXR dataset
# ================================================================
#
# ➊   Uses ImageNet weights, input 224×224×3.
# ➋   Correct MobileNetV2 preprocessing (scales pixels to [-1, 1]).
# ➌   Two-phase training:
#        · Phase-1: head only, base frozen
#        · Phase-2: unfreeze last 30 non-BatchNorm layers
# ➍   Mixed-precision enabled (float16 compute, float32 logits)
# ➎   Checkpoints best weights each fold so you can resume.
# ➏   Evaluates on *clean* Train, Validation, Test generators.
# ➐   Appends metrics to the existing results.csv.
# ---------------------------------------------------------------

import os, time, pickle, tensorflow as tf, pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# ─── mixed precision ────────────────────────────────────────────
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

# ─── hyper-parameters ───────────────────────────────────────────
IMG_SIZE          = (224, 224)
BATCH_SIZE        = 64          # fits easily on T4/L4; raise if you have spare VRAM
EPOCHS_FROZEN     = 10
EPOCHS_FINE_TUNE  = 20
UNFREEZE_LAYERS   = 30          # last N layers to train (skip BatchNorm)
RESULTS_CSV       = "/content/drive/MyDrive/saved_models/results.csv"
TECHNIQUE_NAME    = "MobileNetV2"

# ─── data generators (MobileNetV2 needs [-1,1] scaling) ─────────
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=10, zoom_range=0.1,
    width_shift_range=0.1, height_shift_range=0.1,
    horizontal_flip=True)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# ─── load / init results dataframe ──────────────────────────────
if os.path.exists(RESULTS_CSV):
    results_df = pd.read_csv(RESULTS_CSV)
else:
    results_df = pd.DataFrame(columns=[
        "Technique", "Set", "Fold", "Time(min)", "Epochs Trained",
        "Accuracy", "Precision", "Recall", "F1 Score"])

# ─── model-builder function ─────────────────────────────────────
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

def build_model(num_classes: int):
    base = MobileNetV2(include_top=False,
                       weights='imagenet',
                       input_shape=IMG_SIZE + (3,))
    base.trainable = False                       # phase-1: frozen

    inputs = tf.keras.Input(shape=IMG_SIZE + (3,))
    x = base(inputs, training=False)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.3)(x)
    outputs = Dense(num_classes, activation='softmax', dtype='float32')(x)

    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(1e-4),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model, base

# ─── helper: evaluation & CSV append ────────────────────────────
def evaluate_and_log(set_name, generator, fold_idx, elapsed, epochs_done):
    y_true = generator.classes
    y_pred = model.predict(generator, verbose=0).argmax(axis=1)

    new_row = {
        "Technique"     : TECHNIQUE_NAME,
        "Set"           : set_name,
        "Fold"          : fold_idx,
        "Time(min)"     : round(elapsed, 2),
        "Epochs Trained": epochs_done,
        "Accuracy"      : accuracy_score(y_true, y_pred),
        "Precision"     : precision_score(y_true, y_pred, average='weighted'),
        "Recall"        : recall_score(y_true, y_pred, average='weighted'),
        "F1 Score"      : f1_score(y_true, y_pred, average='weighted')
    }
    global results_df
    results_df = pd.concat([results_df, pd.DataFrame([new_row])],
                           ignore_index=True)

# ─── callbacks template ─────────────────────────────────────────
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
def make_callbacks(fold_idx):
    ckpt_path = f"/content/drive/MyDrive/saved_models/tl_mnv2_fold{fold_idx}_best.weights.h5"
    return [
        EarlyStopping(monitor='val_loss', patience=3,
                      restore_best_weights=True, verbose=1),
        ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                          patience=2, verbose=1),
        ModelCheckpoint(ckpt_path, save_best_only=False,
                        save_weights_only=True, monitor='val_loss', verbose=1)
    ]

# ─── 4-fold cross-validation loop (uses your existing “folds”) ──
for fold_idx, (train_df, test_df) in enumerate(folds, start=1):
    print(f"\n🚀  Fold {fold_idx} / {len(folds)}\n")

    # split off 10 % validation
    train_split_df, val_split_df = train_test_split(
        train_df, test_size=0.10, stratify=train_df['labels'], random_state=42)

    # generators
    train_gen = train_datagen.flow_from_dataframe(
        train_split_df, x_col='filepaths', y_col='labels',
        target_size=IMG_SIZE, batch_size=BATCH_SIZE,
        class_mode='categorical', shuffle=True)

    val_gen = val_datagen.flow_from_dataframe(
        val_split_df, x_col='filepaths', y_col='labels',
        target_size=IMG_SIZE, batch_size=BATCH_SIZE,
        class_mode='categorical', shuffle=False)

    test_gen = val_datagen.flow_from_dataframe(
        test_df, x_col='filepaths', y_col='labels',
        target_size=IMG_SIZE, batch_size=BATCH_SIZE,
        class_mode='categorical', shuffle=False)

    # clean (non-augmented) train generator for metrics
    train_clean_gen = val_datagen.flow_from_dataframe(
        train_df, x_col='filepaths', y_col='labels',
        target_size=IMG_SIZE, batch_size=BATCH_SIZE,
        class_mode='categorical', shuffle=False)

    model, base_model = build_model(len(train_gen.class_indices))

    # ── phase-1: head only ──────────────────────────────────────
    start_time = time.time()
    history1 = model.fit(
        train_gen, validation_data=val_gen,
        epochs=EPOCHS_FROZEN, callbacks=make_callbacks(fold_idx), verbose=1)

    # ── phase-2: fine-tune last N conv layers (skip BatchNorm) ──
    opened = 0
    for layer in reversed(base_model.layers):
        if opened >= UNFREEZE_LAYERS: break
        if not isinstance(layer, tf.keras.layers.BatchNormalization):
            layer.trainable = True
            opened += 1

    model.compile(optimizer=Adam(1e-5),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    start_ft = len(history1.history['loss'])
    history2 = model.fit(
        train_gen, validation_data=val_gen,
        initial_epoch=start_ft, epochs=start_ft + EPOCHS_FINE_TUNE,
        callbacks=make_callbacks(fold_idx), verbose=1)

    total_epochs = len(history1.history['loss']) + len(history2.history['loss'])
    elapsed_min  = (time.time() - start_time) / 60

    # ── save merged history & final model ───────────────────────
    hist = {k: history1.history[k] + history2.history[k] for k in history1.history}
    with open(f"/content/drive/MyDrive/saved_models/mnv2_history_fold{fold_idx}.pkl", "wb") as f:
        pickle.dump(hist, f)
    model.save(f"/content/drive/MyDrive/saved_models/mnv2_fold{fold_idx}.h5")

    # ── evaluation & CSV logging ────────────────────────────────
    evaluate_and_log("Train-clean", train_clean_gen, fold_idx, elapsed_min, total_epochs)
    evaluate_and_log("Validation",   val_gen,       fold_idx, elapsed_min, total_epochs)
    evaluate_and_log("Test",         test_gen,      fold_idx, elapsed_min, total_epochs)

    results_df.to_csv(RESULTS_CSV, index=False)
    print(f"✅  Fold {fold_idx} done — results appended.\n")


In [ ]:
#DenseNet121

import os, time, pickle, tensorflow as tf, pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# ─── mixed precision ────────────────────────────────────────────
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

# ─── hyper-parameters ───────────────────────────────────────────
IMG_SIZE          = (224, 224)
BATCH_SIZE        = 48          # fits easily on T4/L4; raise if you have spare VRAM
EPOCHS_FROZEN     = 10
EPOCHS_FINE_TUNE  = 20
UNFREEZE_LAYERS   = 100         # last N layers to train (skip BatchNorm)
RESULTS_CSV       = "/content/drive/MyDrive/saved_models/results.csv"
TECHNIQUE_NAME    = "DenseNet121"

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.densenet import DenseNet121, preprocess_input


train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=10, zoom_range=0.1,
    width_shift_range=0.1, height_shift_range=0.1,
    horizontal_flip=True)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# ─── load / init results dataframe ──────────────────────────────
if os.path.exists(RESULTS_CSV):
    results_df = pd.read_csv(RESULTS_CSV)
else:
    results_df = pd.DataFrame(columns=[
        "Technique", "Set", "Fold", "Time(min)", "Epochs Trained",
        "Accuracy", "Precision", "Recall", "F1 Score"])

# ─── model-builder function ─────────────────────────────────────
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

def build_model(num_classes: int):
    base = DenseNet121(include_top=False,
                       weights='imagenet',
                       input_shape=IMG_SIZE + (3,))
    base.trainable = False                       # phase-1: frozen

    inputs = tf.keras.Input(shape=IMG_SIZE + (3,))
    x = base(inputs, training=False)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.3)(x)
    outputs = Dense(num_classes, activation='softmax', dtype='float32')(x)

    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(1e-4),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model, base

# ─── helper: evaluation & CSV append ────────────────────────────
def evaluate_and_log(set_name, generator, fold_idx, elapsed, epochs_done):
    y_true = generator.classes
    y_pred = model.predict(generator, verbose=0).argmax(axis=1)

    new_row = {
        "Technique"     : TECHNIQUE_NAME,
        "Set"           : set_name,
        "Fold"          : fold_idx,
        "Time(min)"     : round(elapsed, 2),
        "Epochs Trained": epochs_done,
        "Accuracy"      : accuracy_score(y_true, y_pred),
        "Precision"     : precision_score(y_true, y_pred, average='weighted'),
        "Recall"        : recall_score(y_true, y_pred, average='weighted'),
        "F1 Score"      : f1_score(y_true, y_pred, average='weighted')
    }
    global results_df
    results_df = pd.concat([results_df, pd.DataFrame([new_row])],
                           ignore_index=True)

# ─── callbacks template ─────────────────────────────────────────
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
def make_callbacks(fold_idx):
    ckpt_path = f"/content/drive/MyDrive/saved_models/tl_d121_fold{fold_idx}_best.weights.h5"
    return [
        EarlyStopping(monitor='val_loss', patience=3,
                      restore_best_weights=True, verbose=1),
        ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                          patience=2, verbose=1),
        ModelCheckpoint(ckpt_path, save_best_only=False,
                        save_weights_only=True, monitor='val_loss', verbose=1)
    ]

# ─── 4-fold cross-validation loop (uses your existing “folds”) ──
for fold_idx, (train_df, test_df) in enumerate(folds, start=1):
    print(f"\n🚀  Fold {fold_idx} / {len(folds)}\n")

    # split off 10 % validation
    train_split_df, val_split_df = train_test_split(
        train_df, test_size=0.10, stratify=train_df['labels'], random_state=42)

    # generators
    train_gen = train_datagen.flow_from_dataframe(
        train_split_df, x_col='filepaths', y_col='labels',
        target_size=IMG_SIZE, batch_size=BATCH_SIZE,
        class_mode='categorical', shuffle=True)

    val_gen = val_datagen.flow_from_dataframe(
        val_split_df, x_col='filepaths', y_col='labels',
        target_size=IMG_SIZE, batch_size=BATCH_SIZE,
        class_mode='categorical', shuffle=False)

    test_gen = val_datagen.flow_from_dataframe(
        test_df, x_col='filepaths', y_col='labels',
        target_size=IMG_SIZE, batch_size=BATCH_SIZE,
        class_mode='categorical', shuffle=False)

    # clean (non-augmented) train generator for metrics
    train_clean_gen = val_datagen.flow_from_dataframe(
        train_df, x_col='filepaths', y_col='labels',
        target_size=IMG_SIZE, batch_size=BATCH_SIZE,
        class_mode='categorical', shuffle=False)

    model, base_model = build_model(len(train_gen.class_indices))

    # ── phase-1: head only ──────────────────────────────────────
    start_time = time.time()
    history1 = model.fit(
        train_gen, validation_data=val_gen,
        epochs=EPOCHS_FROZEN, callbacks=make_callbacks(fold_idx), verbose=1)

    # ── phase-2: fine-tune last N conv layers (skip BatchNorm) ──
    opened = 0
    for layer in reversed(base_model.layers):
        if opened >= UNFREEZE_LAYERS: break
        if not isinstance(layer, tf.keras.layers.BatchNormalization):
            layer.trainable = True
            opened += 1

    model.compile(optimizer=Adam(1e-5),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    start_ft = len(history1.history['loss'])
    history2 = model.fit(
        train_gen, validation_data=val_gen,
        initial_epoch=start_ft, epochs=start_ft + EPOCHS_FINE_TUNE,
        callbacks=make_callbacks(fold_idx), verbose=1)

    total_epochs = len(history1.history['loss']) + len(history2.history['loss'])
    elapsed_min  = (time.time() - start_time) / 60

    # ── save merged history & final model ───────────────────────
    hist = {k: history1.history[k] + history2.history[k] for k in history1.history}
    with open(f"/content/drive/MyDrive/saved_models/mnv2_history_fold{fold_idx}.pkl", "wb") as f:
        pickle.dump(hist, f)
    model.save(f"/content/drive/MyDrive/saved_models/mnv2_fold{fold_idx}.h5")

    # ── evaluation & CSV logging ────────────────────────────────
    evaluate_and_log("Train-clean", train_clean_gen, fold_idx, elapsed_min, total_epochs)
    evaluate_and_log("Validation",   val_gen,       fold_idx, elapsed_min, total_epochs)
    evaluate_and_log("Test",         test_gen,      fold_idx, elapsed_min, total_epochs)

    results_df.to_csv(RESULTS_CSV, index=False)
    print(f"✅  Fold {fold_idx} done — results appended.\n")


In [ ]:
# === EfficientNetB0 Transfer Learning with 4-Fold CV ===

import os, time, pickle, tensorflow as tf, pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# ─── mixed precision (optional, saves VRAM on Colab) ───
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

# ─── hyperparameters & paths ───────────────────────────
IMG_SIZE = (224, 224)
BATCH_SIZE = 64
EPOCHS_FROZEN = 10
EPOCHS_FINE_TUNE = 20
UNFREEZE_LAYERS = 30
RESULTS_CSV = "/content/drive/MyDrive/saved_models/results.csv"
TECHNIQUE_NAME = "EfficientNetB0"

# ─── Data generators ───────────────────────────────────
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import preprocess_input

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=10, zoom_range=0.1,
    width_shift_range=0.1, height_shift_range=0.1,
    horizontal_flip=True)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# ─── Load or init results CSV ──────────────────────────
if os.path.exists(RESULTS_CSV):
    results_df = pd.read_csv(RESULTS_CSV)
else:
    results_df = pd.DataFrame(columns=[
        "Technique", "Set", "Fold", "Time(min)", "Epochs Trained",
        "Accuracy", "Precision", "Recall", "F1 Score"
    ])

# ─── Model builder ─────────────────────────────────────
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

def build_model(num_classes: int):
    base = EfficientNetB0(include_top=False, weights="imagenet", input_shape=IMG_SIZE + (3,))
    base.trainable = False

    inputs = tf.keras.Input(shape=IMG_SIZE + (3,))
    x = base(inputs, training=False)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    x = Dense(128, activation="relu")(x)
    x = Dropout(0.3)(x)
    outputs = Dense(num_classes, activation="softmax", dtype="float32")(x)

    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(1e-4), loss="categorical_crossentropy", metrics=["accuracy"])
    return model, base

# ─── Evaluation helper ─────────────────────────────────
def evaluate_and_log(set_name, generator, fold_idx, elapsed, epochs_done):
    y_true = generator.classes
    y_pred = model.predict(generator, verbose=0).argmax(axis=1)

    new_row = {
        "Technique": TECHNIQUE_NAME,
        "Set": set_name,
        "Fold": fold_idx,
        "Time(min)": round(elapsed, 2),
        "Epochs Trained": epochs_done,
        "Accuracy": accuracy_score(y_true, y_pred),
        "Precision": precision_score(y_true, y_pred, average="weighted"),
        "Recall": recall_score(y_true, y_pred, average="weighted"),
        "F1 Score": f1_score(y_true, y_pred, average="weighted")
    }
    global results_df
    results_df = pd.concat([results_df, pd.DataFrame([new_row])], ignore_index=True)

# ─── Callbacks ─────────────────────────────────────────
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

def make_callbacks(fold_idx):
    ckpt_path = f"/content/drive/MyDrive/saved_models/effb0_fold{fold_idx}_best.weights.h5"
    return [
        EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True, verbose=1),
        ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=2, verbose=1),
        ModelCheckpoint(ckpt_path, save_best_only=False, save_weights_only=True,
                        monitor="val_loss", verbose=1)
    ]

# ─── Training loop ─────────────────────────────────────
for fold_idx, (train_df, test_df) in enumerate(folds, start=1):
    print(f"\n🚀 Fold {fold_idx} / {len(folds)} — EfficientNetB0\n")

    # 10% validation split from training
    train_split_df, val_split_df = train_test_split(
        train_df, test_size=0.10, stratify=train_df["labels"], random_state=42)

    # Data generators
    train_gen = train_datagen.flow_from_dataframe(
        train_split_df, x_col="filepaths", y_col="labels",
        target_size=IMG_SIZE, batch_size=BATCH_SIZE,
        class_mode="categorical", shuffle=True)

    val_gen = val_datagen.flow_from_dataframe(
        val_split_df, x_col="filepaths", y_col="labels",
        target_size=IMG_SIZE, batch_size=BATCH_SIZE,
        class_mode="categorical", shuffle=False)

    test_gen = val_datagen.flow_from_dataframe(
        test_df, x_col="filepaths", y_col="labels",
        target_size=IMG_SIZE, batch_size=BATCH_SIZE,
        class_mode="categorical", shuffle=False)

    train_clean_gen = val_datagen.flow_from_dataframe(
        train_df, x_col="filepaths", y_col="labels",
        target_size=IMG_SIZE, batch_size=BATCH_SIZE,
        class_mode="categorical", shuffle=False)

    model, base_model = build_model(len(train_gen.class_indices))

    # Phase 1: frozen base
    start_time = time.time()
    history1 = model.fit(train_gen, validation_data=val_gen,
                         epochs=EPOCHS_FROZEN, callbacks=make_callbacks(fold_idx), verbose=1)

    # Phase 2: fine-tune last N layers (skip BatchNorm)
    opened = 0
    for layer in reversed(base_model.layers):
        if opened >= UNFREEZE_LAYERS:
            break
        if not isinstance(layer, tf.keras.layers.BatchNormalization):
            layer.trainable = True
            opened += 1

    model.compile(optimizer=Adam(1e-5), loss="categorical_crossentropy", metrics=["accuracy"])
    start_ft = len(history1.history["loss"])
    history2 = model.fit(train_gen, validation_data=val_gen,
                         initial_epoch=start_ft, epochs=start_ft + EPOCHS_FINE_TUNE,
                         callbacks=make_callbacks(fold_idx), verbose=1)

    # Save
    total_epochs = len(history1.history["loss"]) + len(history2.history["loss"])
    elapsed_min = (time.time() - start_time) / 60

    hist = {k: history1.history[k] + history2.history[k] for k in history1.history}
    with open(f"/content/drive/MyDrive/saved_models/effb0_history_fold{fold_idx}.pkl", "wb") as f:
        pickle.dump(hist, f)

    model.save(f"/content/drive/MyDrive/saved_models/effb0_fold{fold_idx}.h5")

    # Evaluation
    evaluate_and_log("Train-clean", train_clean_gen, fold_idx, elapsed_min, total_epochs)
    evaluate_and_log("Validation", val_gen, fold_idx, elapsed_min, total_epochs)
    evaluate_and_log("Test", test_gen, fold_idx, elapsed_min, total_epochs)

    results_df.to_csv(RESULTS_CSV, index=False)
    print(f"✅ Fold {fold_idx} complete. Results saved.\n")


In [ ]:
# ================================================================
# Transfer-learning baseline: VGG-19 on 4-fold CXR dataset
# ================================================================
#
# ➊   ImageNet weights, input 224×224×3.
# ➋   Proper VGG19 preprocessing (mean-subtraction, RGB->BGR).
# ➌   Two-phase training:
#        · Phase-1: head only, base frozen
#        · Phase-2: unfreeze last 8 conv layers
# ➍   Mixed precision enabled (float16 compute, float32 logits).
# ➎   Checkpoints best weights each fold so you can resume.
# ➏   Evaluates on clean Train, Validation, Test generators.
# ➐   Appends metrics to the existing results.csv.
# ---------------------------------------------------------------

import os, time, pickle, tensorflow as tf, pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# ─── mixed precision ────────────────────────────────────────────
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

# ─── hyper-parameters ───────────────────────────────────────────
IMG_SIZE          = (224, 224)
BATCH_SIZE        = 32            # VGG19 is heavy; 32 fits on Colab T4/L4
EPOCHS_FROZEN     = 5
EPOCHS_FINE_TUNE  = 15
UNFREEZE_LAYERS   = 8             # last 8 conv layers
RESULTS_CSV       = "/content/drive/MyDrive/saved_models/results.csv"
TECHNIQUE_NAME    = "TL-VGG19"

# ─── data generators (VGG19 preprocess) ─────────────────────────
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg19 import preprocess_input

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=10, zoom_range=0.1,
    width_shift_range=0.1, height_shift_range=0.1,
    horizontal_flip=True)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# ─── load / init results dataframe ──────────────────────────────
if os.path.exists(RESULTS_CSV):
    results_df = pd.read_csv(RESULTS_CSV)
else:
    results_df = pd.DataFrame(columns=[
        "Technique", "Set", "Fold", "Time(min)", "Epochs Trained",
        "Accuracy", "Precision", "Recall", "F1 Score"])

# ─── model-builder function ─────────────────────────────────────
from tensorflow.keras.applications import VGG19
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

def build_model(num_classes: int):
    base = VGG19(include_top=False,
                 weights='imagenet',
                 input_shape=IMG_SIZE + (3,))
    base.trainable = False                        # phase-1: frozen

    inputs = tf.keras.Input(shape=IMG_SIZE + (3,))
    x = base(inputs, training=False)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.3)(x)
    outputs = Dense(num_classes, activation='softmax', dtype='float32')(x)

    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(1e-4),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model, base

# ─── evaluation helper ──────────────────────────────────────────
def evaluate_and_log(set_name, generator, fold_idx, elapsed, epochs_done):
    y_true = generator.classes
    y_pred = model.predict(generator, verbose=0).argmax(axis=1)

    new_row = {
        "Technique"     : TECHNIQUE_NAME,
        "Set"           : set_name,
        "Fold"          : fold_idx,
        "Time(min)"     : round(elapsed, 2),
        "Epochs Trained": epochs_done,
        "Accuracy"      : accuracy_score(y_true, y_pred),
        "Precision"     : precision_score(y_true, y_pred, average='weighted'),
        "Recall"        : recall_score(y_true, y_pred, average='weighted'),
        "F1 Score"      : f1_score(y_true, y_pred, average='weighted')
    }
    global results_df
    results_df = pd.concat([results_df, pd.DataFrame([new_row])],
                           ignore_index=True)

# ─── callbacks template ─────────────────────────────────────────
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
def make_callbacks(fold_idx):
    ckpt_path = f"/content/drive/MyDrive/saved_models/tl_vgg19_fold{fold_idx}_best.weights.h5"
    return [
        EarlyStopping(monitor='val_loss', patience=3,
                      restore_best_weights=True, verbose=1),
        ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                          patience=2, verbose=1),
        ModelCheckpoint(ckpt_path, save_best_only=True,
                        save_weights_only=True, monitor='val_loss', verbose=1)
    ]

# ─── 4-fold cross-validation loop (uses your existing “folds”) ──
for fold_idx, (train_df, test_df) in enumerate(folds, start=1):
    print(f"\n🚀  Fold {fold_idx} / {len(folds)}\n")

    # split off 10 % validation
    train_split_df, val_split_df = train_test_split(
        train_df, test_size=0.10, stratify=train_df['labels'], random_state=42)

    # generators
    train_gen = train_datagen.flow_from_dataframe(
        train_split_df, x_col='filepaths', y_col='labels',
        target_size=IMG_SIZE, batch_size=BATCH_SIZE,
        class_mode='categorical', shuffle=True)

    val_gen = val_datagen.flow_from_dataframe(
        val_split_df, x_col='filepaths', y_col='labels',
        target_size=IMG_SIZE, batch_size=BATCH_SIZE,
        class_mode='categorical', shuffle=False)

    test_gen = val_datagen.flow_from_dataframe(
        test_df, x_col='filepaths', y_col='labels',
        target_size=IMG_SIZE, batch_size=BATCH_SIZE,
        class_mode='categorical', shuffle=False)

    train_clean_gen = val_datagen.flow_from_dataframe(
        train_df, x_col='filepaths', y_col='labels',
        target_size=IMG_SIZE, batch_size=BATCH_SIZE,
        class_mode='categorical', shuffle=False)
    num_classes = len(train_gen.class_indices)   # robust across TF versions
    model, base_model = build_model(num_classes)



    # ── phase-1: head only ──────────────────────────────────────
    start_time = time.time()
    history1 = model.fit(
        train_gen, validation_data=val_gen,
        epochs=EPOCHS_FROZEN, callbacks=make_callbacks(fold_idx), verbose=1)

    # ── phase-2: fine-tune last N conv layers ───────────────────
    opened = 0
    for layer in reversed(base_model.layers):
        if opened >= UNFREEZE_LAYERS: break
        if 'conv' in layer.name:
            layer.trainable = True
            opened += 1

    model.compile(optimizer=Adam(1e-5),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    start_ft = len(history1.history['loss'])
    history2 = model.fit(
        train_gen, validation_data=val_gen,
        initial_epoch=start_ft, epochs=start_ft + EPOCHS_FINE_TUNE,
        callbacks=make_callbacks(fold_idx), verbose=1)

    total_epochs = len(history1.history['loss']) + len(history2.history['loss'])
    elapsed_min  = (time.time() - start_time) / 60

    # ── save history & model ────────────────────────────────────
    hist = {k: history1.history[k] + history2.history[k] for k in history1.history}
    with open(f"/content/drive/MyDrive/saved_models/vgg19_history_fold{fold_idx}.pkl", "wb") as f:
        pickle.dump(hist, f)
    model.save(f"/content/drive/MyDrive/saved_models/vgg19_fold{fold_idx}.h5")

    # ── evaluation & CSV logging ────────────────────────────────
    evaluate_and_log("Train-clean", train_clean_gen, fold_idx, elapsed_min, total_epochs)
    evaluate_and_log("Validation",   val_gen,       fold_idx, elapsed_min, total_epochs)
    evaluate_and_log("Test",         test_gen,      fold_idx, elapsed_min, total_epochs)

    results_df.to_csv(RESULTS_CSV, index=False)
    print(f"✅  Fold {fold_idx} done — results appended.\n")


In [ ]:
import pickle
import matplotlib.pyplot as plt
#Plot accuracy and Loss Curves for 1 fold
# Load history
with open('/content/drive/MyDrive/saved_models/mnv2_history_fold3.pkl', 'rb') as f: #changed based on CNN and fold
    history = pickle.load(f)

# Plot loss
plt.plot(history['loss'], label='Train Loss')
plt.plot(history['val_loss'], label='Val Loss')
plt.title('Fold 3 - Loss Curve')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid()
plt.show()

# Plot accuracy
plt.plot(history['accuracy'], label='Train Acc')
plt.plot(history['val_accuracy'], label='Val Acc')
plt.title('Fold 3 - Accuracy Curve')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid()
plt.show()


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=10, zoom_range=0.1,
    width_shift_range=0.1, height_shift_range=0.1,
    horizontal_flip=True)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [ ]:
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import numpy as np

# Load test generator (you must re-create it like before)
test_gen = val_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='filepaths',
    y_col='labels',
    target_size=(224, 224),
    class_mode='categorical',
    shuffle=False,
    batch_size=64
)

# Load model
model = load_model('/content/drive/MyDrive/saved_models/mnv2_fold1.h5')

# Predict
y_probs = model.predict(test_gen, verbose=1)
y_pred = np.argmax(y_probs, axis=1)
y_true = test_gen.classes

# Plot Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(cm, display_labels=test_gen.class_indices.keys())
disp.plot(cmap='Blues', xticks_rotation=45)
plt.title('Confusion Matrix - Fold 1')
plt.grid(False)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 📊 Plot class distributions for ALL folds
for fold_idx, (train_df, test_df) in enumerate(folds):
    print(f"\n🔵 Class Distribution for Fold {fold_idx + 1}\n")

    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    # 🧪 Training set
    sns.countplot(ax=axes[0], data=train_df, x='labels', order=sorted(train_df['labels'].unique()))
    axes[0].set_title(f"Training Set - Fold {fold_idx + 1}")
    axes[0].set_xlabel("Class Label")
    axes[0].set_ylabel("Number of Images")
    axes[0].grid(True)

    # 🧪 Testing set
    sns.countplot(ax=axes[1], data=test_df, x='labels', order=sorted(test_df['labels'].unique()))
    axes[1].set_title(f"Test Set - Fold {fold_idx + 1}")
    axes[1].set_xlabel("Class Label")
    axes[1].set_ylabel("Number of Images")
    axes[1].grid(True)

    plt.tight_layout()
    plt.show()



In [ ]:
#UPDATED
#CNN FROM SCRATCH
import time
import pickle
import os
import pandas as pd
from sklearn.model_selection import StratifiedKFold, train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# CSV path to store results
results_csv_path = "/content/drive/MyDrive/saved_models/results.csv"
technique_name = "CNN from scratch"

# Initialize results DataFrame
if os.path.exists(results_csv_path):
    results_df = pd.read_csv(results_csv_path)
else:
    results_df = pd.DataFrame(columns=[
        "Technique", "Set", "Fold", "Time(min)", "Epochs Trained",
        "Accuracy", "Precision", "Recall", "F1 Score"
    ])

# 🔵 Loop through each fold
for fold_idx, (train_df, test_df) in enumerate(folds):
    print(f"\n🔵 Fold {fold_idx + 1} starting...\n")

    # ➗ Split train_df into 90% train and 10% validation
    train_split_df, val_split_df = train_test_split(
        train_df,
        test_size=0.1,
        stratify=train_df['labels'],
        random_state=42
    )

    # 🔹 Create Generators
    train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_split_df,
        x_col='filepaths',
        y_col='labels',
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=True
    )

    val_generator = val_datagen.flow_from_dataframe(
        dataframe=val_split_df,
        x_col='filepaths',
        y_col='labels',
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=False
    )

    test_generator = val_datagen.flow_from_dataframe(
        dataframe=test_df,
        x_col='filepaths',
        y_col='labels',
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=False
    )

    # 🔹 Create New Model
    model = create_model()

    # 🔹 Callbacks
    early_stop = EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True,
        verbose=1
    )

    checkpoint = ModelCheckpoint(
        filepath=f"/content/drive/MyDrive/saved_models/cnn_fold_{fold_idx + 1}_checkpoint.h5",
        monitor='val_loss',
        save_best_only=False,
        verbose=1
    )

    # ⏱️ Start training
    start_time = time.time()

    history = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=40,
        callbacks=[early_stop, checkpoint],
        verbose=1
    )

    end_time = time.time()
    elapsed_time = (end_time - start_time) / 60
    epochs_trained = len(history.history['loss'])

    # 💾 Save Final Model & History
    model.save(f"/content/drive/MyDrive/saved_models/cnn_fold_{fold_idx + 1}.h5")
    with open(f"/content/drive/MyDrive/saved_models/history_fold_{fold_idx + 1}.pkl", 'wb') as f:
        pickle.dump(history.history, f)

    # 🧮 Function to evaluate and append to results_df
    def evaluate_and_log(set_name, generator):
        global results_df
        y_true = generator.classes
        y_probs = model.predict(generator, verbose=0)
        y_pred = y_probs.argmax(axis=1)

        acc = accuracy_score(y_true, y_pred)
        prec = precision_score(y_true, y_pred, average='weighted')
        rec = recall_score(y_true, y_pred, average='weighted')
        f1 = f1_score(y_true, y_pred, average='weighted')

        new_row = {
            "Technique": technique_name,
            "Set": set_name,
            "Fold": fold_idx + 1,
            "Time(min)": round(elapsed_time, 2),
            "Epochs Trained": epochs_trained,
            "Accuracy": acc,
            "Precision": prec,
            "Recall": rec,
            "F1 Score": f1
        }


        results_df = pd.concat([results_df, pd.DataFrame([new_row])], ignore_index=True)

    # 🔍 Evaluate Train, Validation, and Test
    evaluate_and_log("Train", train_generator)
    evaluate_and_log("Validation", val_generator)
    evaluate_and_log("Test", test_generator)

    # 💾 Save to CSV
    results_df.to_csv(results_csv_path, index=False)
    print(f"\n✅ Fold {fold_idx + 1} results saved to CSV.\n")


In [ ]:
#updated
from tensorflow.keras.models import load_model

# Store accuracies and losses
fold_accuracies = []
fold_losses = []

# Evaluate each fold
for fold_idx, (train_df, test_df) in enumerate(folds):  # ⬅️ CHANGED val_df ➔ test_df
    print(f"\n🔵 Evaluating Fold {fold_idx + 1}...\n")

    # Reload the model
    model = load_model(f"/content/drive/MyDrive/saved_models/effb0_fold_{fold_idx + 1}.h5") #CHANGE cnn_fold_

    # Create test generator (based on test_df)
    test_generator = val_datagen.flow_from_dataframe(
        dataframe=test_df,
        x_col='filepaths',
        y_col='labels',
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=False
    )

    # Evaluate on the test generator
    test_loss, test_acc = model.evaluate(test_generator, verbose=1)

    print(f"🎯 Fold {fold_idx + 1} Test Accuracy: {test_acc:.4f}")
    print(f"🎯 Fold {fold_idx + 1} Test Loss: {test_loss:.4f}")

    # Save results
    fold_accuracies.append(test_acc)
    fold_losses.append(test_loss)

# Calculate average results
avg_accuracy = sum(fold_accuracies) / len(fold_accuracies)
avg_loss = sum(fold_losses) / len(fold_losses)

print("\n📈 Average over all Folds:")
print(f"✅ Average Test Accuracy: {avg_accuracy:.4f}")
print(f"✅ Average Test Loss: {avg_loss:.4f}")


In [ ]:
import matplotlib.pyplot as plt
import pickle  # Needed to load the histories

# Lists to collect history for all folds
histories = []

# 🔵 Load each fold's saved history
for fold_idx in range(4):
    print(f"\n🔵 Loading Training History for Fold {fold_idx + 1}...\n")

    with open(f"/content/drive/MyDrive/saved_models/history_fold_{fold_idx + 1}.pkl", 'rb') as f:
        history_data = pickle.load(f)
        histories.append(history_data)

# 🖼️ Now plot for each fold
for fold_idx, history_data in enumerate(histories):
    print(f"\n🖼️ Plotting Fold {fold_idx + 1}...\n")

    # Accuracy Plot
    plt.figure(figsize=(8, 5))
    plt.plot(history_data['accuracy'], label='Train Accuracy')
    plt.plot(history_data['val_accuracy'], label='Validation Accuracy')
    plt.title(f'Model Accuracy over Epochs - Fold {fold_idx + 1}')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)
    plt.show()

    # Loss Plot
    plt.figure(figsize=(8, 5))
    plt.plot(history_data['loss'], label='Train Loss')
    plt.plot(history_data['val_loss'], label='Validation Loss')
    plt.title(f'Model Loss over Epochs - Fold {fold_idx + 1}')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    plt.show()



In [ ]:
#updated
#PLOT CONFUSION MATRIXES
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

# 🟡 Loop through each fold
for fold_idx, (train_df, test_df) in enumerate(folds):  # ✅ val_df ➜ test_df
    print(f"\n🔵 Fold {fold_idx + 1} Confusion Matrix and Classification Report:\n")

    # Create test generator
    test_generator = val_datagen.flow_from_dataframe(  # ✅ val_generator ➜ test_generator
        dataframe=test_df,
        x_col='filepaths',
        y_col='labels',
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=False
    )

    # Load the saved model
    model = load_model(f"/content/drive/MyDrive/saved_models/tl_d121_fold_{fold_idx + 1}.h5")

    # Predict
    y_pred_probs = model.predict(test_generator, verbose=1)
    y_pred_classes = np.argmax(y_pred_probs, axis=1)

    # True labels
    y_true = test_generator.classes
    class_labels = list(test_generator.class_indices.keys())

    # Confusion Matrix
    cm = confusion_matrix(y_true, y_pred_classes)

    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_labels, yticklabels=class_labels)
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.title(f'Confusion Matrix - Fold {fold_idx + 1}')
    plt.show()

    # Classification Report
    report = classification_report(y_true, y_pred_classes, target_names=class_labels)
    print(report)


In [ ]:
# ─── Data generators ─────────────────────────────────── #used for effnetb0 conf matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import preprocess_input

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=10, zoom_range=0.1,
    width_shift_range=0.1, height_shift_range=0.1,
    horizontal_flip=True)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
from sklearn.model_selection import StratifiedKFold

K = 4
skf = StratifiedKFold(n_splits=K, shuffle=True, random_state=42)
folds = []

for train_idx, test_idx in skf.split(df['filepaths'], df['labels']):
    train_data = df.iloc[train_idx].reset_index(drop=True)
    test_data = df.iloc[test_idx].reset_index(drop=True)
    folds.append((train_data, test_data))


In [ ]:
#EfficientNetB0 Confusion Matrix
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
import tensorflow as tf
from tensorflow.keras import mixed_precision

# Dummy placeholder for 'Cast'
class DummyCast(tf.keras.layers.Layer):
    def call(self, inputs):
        return inputs

# Register both required components
custom_objects = {
    'Cast': DummyCast,
    'Policy': mixed_precision.Policy
}

# 🟡 Loop through each fold
for fold_idx, (train_df, test_df) in enumerate(folds):
    print(f"\n🔵 Fold {fold_idx + 1} Confusion Matrix and Classification Report:\n")

    # Create test generator
    test_generator = val_datagen.flow_from_dataframe(
        dataframe=test_df,
        x_col='filepaths',
        y_col='labels',
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=False
    )

    # 🔄 Load the correct model for the current fold inside the loop
    with tf.keras.utils.custom_object_scope(custom_objects):
        model = load_model(f"/content/drive/MyDrive/saved_models/effb0_fold{fold_idx + 1}.h5")

    # Predict
    y_pred_probs = model.predict(test_generator, verbose=1)
    y_pred_classes = np.argmax(y_pred_probs, axis=1)

    # True labels
    y_true = test_generator.classes
    class_labels = list(test_generator.class_indices.keys())

    # Confusion Matrix
    cm = confusion_matrix(y_true, y_pred_classes)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_labels, yticklabels=class_labels)
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.title(f'Confusion Matrix - Fold {fold_idx + 1}')
    plt.show()

    # Classification Report
    report = classification_report(y_true, y_pred_classes, target_names=class_labels)
    print(report)


In [ ]:
#mobile net v2 CONFUSION MATRIX
for fold_idx, (_, test_df) in enumerate(folds):
    print(f"\n🔵 Fold {fold_idx + 1} Confusion Matrix and Classification Report:\n")

    test_gen = val_datagen.flow_from_dataframe(
        dataframe=test_df,
        x_col='filepaths',
        y_col='labels',
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=False
    )

    model_path = f"/content/drive/MyDrive/saved_models/mnv2_fold{fold_idx + 1}.h5"

    # 👇 Fix loading error for mixed precision
    with tf.keras.utils.custom_object_scope(custom_objects):
        model = load_model(model_path)

    y_pred_probs = model.predict(test_gen, verbose=0)
    y_pred = np.argmax(y_pred_probs, axis=1)
    y_true = test_gen.classes
    class_labels = list(test_gen.class_indices.keys())

    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_labels, yticklabels=class_labels)
    plt.xlabel("Predicted Labels")
    plt.ylabel("True Labels")
    plt.title(f"Confusion Matrix - Fold {fold_idx + 1}")
    plt.show()

    print(classification_report(y_true, y_pred, target_names=class_labels))


In [ ]:
#vgg19 CONFUSION MATRIX

import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
import tensorflow as tf
from tensorflow.keras import mixed_precision

# Dummy placeholder for 'Cast'
class DummyCast(tf.keras.layers.Layer):
    def call(self, inputs):
        return inputs

# Register both required components
custom_objects = {
    'Cast': DummyCast,
    'Policy': mixed_precision.Policy
}

# 🟡 Loop through each fold
for fold_idx, (train_df, test_df) in enumerate(folds):
    print(f"\n🔵 Fold {fold_idx + 1} Confusion Matrix and Classification Report:\n")

    # Create test generator
    test_generator = val_datagen.flow_from_dataframe(
        dataframe=test_df,
        x_col='filepaths',
        y_col='labels',
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=False
    )

    # 🔄 Load the correct model for the current fold inside the loop
    with tf.keras.utils.custom_object_scope(custom_objects):
        model = load_model(f"/content/drive/MyDrive/saved_models/vgg19_fold{fold_idx + 1}.h5")

    # Predict
    y_pred_probs = model.predict(test_generator, verbose=1)
    y_pred_classes = np.argmax(y_pred_probs, axis=1)

    # True labels
    y_true = test_generator.classes
    class_labels = list(test_generator.class_indices.keys())

    # Confusion Matrix
    cm = confusion_matrix(y_true, y_pred_classes)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_labels, yticklabels=class_labels)
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.title(f'Confusion Matrix - Fold {fold_idx + 1}')
    plt.show()

    # Classification Report
    report = classification_report(y_true, y_pred_classes, target_names=class_labels)
    print(report)


In [ ]:
#densenet121 CONFUSION MATRIX

import numpy as np, seaborn as sns, matplotlib.pyplot as plt, tensorflow as tf
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.models import load_model
from tensorflow.keras import mixed_precision

# ── custom-object safety net (needed when model saved with MP) ──
class DummyCast(tf.keras.layers.Layer):
    def call(self, inputs):
        return inputs

custom_objects = {
    "Cast"  : DummyCast,         # for legacy mp layers
    "Policy": mixed_precision.Policy
}

# ── iterate through the four folds ──────────────────────────────
for fold_idx, (_, test_df) in enumerate(folds, start=1):
    print(f"\n🔵  Fold {fold_idx} – Confusion Matrix & Report\n")

    test_gen = val_datagen.flow_from_dataframe(
        dataframe   = test_df,
        x_col       = "filepaths",
        y_col       = "labels",
        target_size = IMG_SIZE,
        batch_size  = BATCH_SIZE,
        class_mode  = "categorical",
        shuffle     = False)

    # load DenseNet-121 model for this fold
    model_path = f"/content/drive/MyDrive/saved_models/mnv2_fold{fold_idx}.h5"
    with tf.keras.utils.custom_object_scope(custom_objects):
        model = load_model(model_path)

    # predictions
    y_prob = model.predict(test_gen, verbose=1)
    y_pred = y_prob.argmax(axis=1)

    # ground-truth
    y_true        = test_gen.classes
    class_labels  = list(test_gen.class_indices.keys())

    # ── confusion matrix ─────────────────────────────────────────
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap="Blues",
                xticklabels=class_labels, yticklabels=class_labels)
    plt.title(f"Confusion Matrix — DenseNet121 — Fold {fold_idx}")
    plt.xlabel("Predicted label");  plt.ylabel("True label")
    plt.tight_layout();  plt.show()

    # ── classification report ───────────────────────────────────
    print(classification_report(y_true, y_pred, target_names=class_labels))


In [ ]:
#PLOT ROC CURVES FOR EACH CLASS (CNN FROM SCRATCH)
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pickle

# Constants
IMG_SIZE = (224, 224)
BATCH_SIZE = 64
NUM_CLASSES = 4
FOLDS = 4
from typing import List

classnames: List[str] = ["COVID", "Lung Opacity", "Normal", "Viral Pneumonia"]

# ImageDataGenerator (same as your CNN from scratch)
val_datagen = ImageDataGenerator(rescale=1./255)

# Loop through each fold
for fold_idx in range(1, FOLDS + 1):
    print(f"\n📊 ROC Curve for Fold {fold_idx}")

    # Get test dataframe
    _, test_df = folds[fold_idx - 1]

    # Load model
    model_path = f"/content/drive/MyDrive/saved_models/cnn_fold_{fold_idx}.h5"
    model = load_model(model_path)

    # Create test generator
    test_gen = val_datagen.flow_from_dataframe(
        dataframe=test_df,
        x_col='filepaths',
        y_col='labels',
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=False
    )

    # True labels and predictions
    y_true = test_gen.classes
    y_true_bin = label_binarize(y_true, classes=list(range(NUM_CLASSES)))

    y_probs = model.predict(test_gen, verbose=0)

    # Plot ROC for each class
    plt.figure(figsize=(8, 6))
    for i in range(NUM_CLASSES):
        fpr, tpr, _ = roc_curve(y_true_bin[:, i], y_probs[:, i])
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, label=f"{classnames[i]} (AUC = {roc_auc:.2f})")

    plt.plot([0, 1], [0, 1], 'k--', lw=1)
    plt.title(f"ROC Curve – Fold {fold_idx} – CNN from Scratch")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.legend(loc="lower right")
    plt.grid(True)
    plt.show()


In [ ]:
#PLOT  ROC CURVES FOR EACH CLASS (MOBILENETV2)
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

# Constants
IMG_SIZE = (224, 224)
BATCH_SIZE = 64
NUM_CLASSES = 4
FOLDS = 4
classnames = ["COVID", "Lung Opacity", "Normal", "Viral Pneumonia"]

# Use correct MobileNetV2 preprocessing
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Model builder (same as training)
def build_model(num_classes: int):
    base = MobileNetV2(include_top=False, weights='imagenet', input_shape=IMG_SIZE + (3,))
    base.trainable = True  # We're just using this to load weights
    inputs = tf.keras.Input(shape=IMG_SIZE + (3,))
    x = base(inputs, training=False)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.3)(x)
    outputs = Dense(num_classes, activation='softmax', dtype='float32')(x)
    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Plot ROC curve for each fold
for fold_idx in range(1, FOLDS + 1):
    print(f"\n📊 ROC Curve for Fold {fold_idx} — MobileNetV2")

    # Get test set from folds
    _, test_df = folds[fold_idx - 1]

    # Generator for test data
    test_gen = val_datagen.flow_from_dataframe(
        dataframe=test_df,
        x_col='filepaths',
        y_col='labels',
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=False
    )

    # Build model and load weights
    model = build_model(NUM_CLASSES)
    weights_path = f"/content/drive/MyDrive/saved_models/tl_mnv2_fold{fold_idx}_best.weights.h5"
    model.load_weights(weights_path)

    # True labels and predicted probabilities
    y_true = test_gen.classes
    y_true_bin = label_binarize(y_true, classes=list(range(NUM_CLASSES)))
    y_probs = model.predict(test_gen, verbose=0)

    # Plot ROC curves per class
    plt.figure(figsize=(8, 6))
    for i in range(NUM_CLASSES):
        fpr, tpr, _ = roc_curve(y_true_bin[:, i], y_probs[:, i])
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, label=f"{classnames[i]} (AUC = {roc_auc:.2f})")

    plt.plot([0, 1], [0, 1], 'k--', lw=1)
    plt.title(f"ROC Curve – Fold {fold_idx} – MobileNetV2")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.legend(loc="lower right")
    plt.grid(True)
    plt.show()


In [ ]:
#EfficientNetb0 ROC CURVES
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import EfficientNetB0, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

# Constants
IMG_SIZE = (224, 224)
BATCH_SIZE = 64
NUM_CLASSES = 4
FOLDS = 4
classnames = ["COVID", "Lung Opacity", "Normal", "Viral Pneumonia"]

# EfficientNetB0 uses its own preprocessing
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Model builder (same as training)
def build_model(num_classes: int):
    base = EfficientNetB0(include_top=False, weights='imagenet', input_shape=IMG_SIZE + (3,))
    base.trainable = True  # Required to allow weight loading
    inputs = tf.keras.Input(shape=IMG_SIZE + (3,))
    x = base(inputs, training=False)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.3)(x)
    outputs = Dense(num_classes, activation='softmax', dtype='float32')(x)
    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Loop through each fold
for fold_idx in range(1, FOLDS + 1):
    print(f"\n📊 ROC Curve for Fold {fold_idx} — EfficientNetB0")

    # Load test set
    _, test_df = folds[fold_idx - 1]

    # Test generator
    test_gen = val_datagen.flow_from_dataframe(
        dataframe=test_df,
        x_col='filepaths',
        y_col='labels',
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=False
    )

    # Build model and load weights
    model = build_model(NUM_CLASSES)
    weights_path = f"/content/drive/MyDrive/saved_models/effb0_fold{fold_idx}_best.weights.h5"
    model.load_weights(weights_path)

    # Predict
    y_true = test_gen.classes
    y_true_bin = label_binarize(y_true, classes=list(range(NUM_CLASSES)))
    y_probs = model.predict(test_gen, verbose=0)

    # Plot ROC curve
    plt.figure(figsize=(8, 6))
    for i in range(NUM_CLASSES):
        fpr, tpr, _ = roc_curve(y_true_bin[:, i], y_probs[:, i])
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, label=f"{classnames[i]} (AUC = {roc_auc:.2f})")

    plt.plot([0, 1], [0, 1], 'k--', lw=1)
    plt.title(f"ROC Curve – Fold {fold_idx} – EfficientNetB0")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.legend(loc="lower right")
    plt.grid(True)
    plt.show()


In [ ]:
#VGG19 ROC CURVES

import numpy as np, matplotlib.pyplot as plt, tensorflow as tf, os
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model

# ─── constants ──────────────────────────────────────────────────
IMG_SIZE     = (224, 224)
BATCH_SIZE   = 32               # matches training batch for VGG-19
NUM_CLASSES  = 4
FOLDS        = 4
CLASSNAMES   = ["COVID", "Lung Opacity", "Normal", "Viral Pneumonia"]

# ─── non-augmented generator for evaluation ─────────────────────
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# ─── model builder (same head used during fine-tuning) ──────────
def build_vgg19_model(num_classes: int = NUM_CLASSES):
    base = VGG19(include_top=False, weights='imagenet',
                 input_shape=IMG_SIZE + (3,))
    base.trainable = True  # required to load unfrozen weights
    inputs  = tf.keras.Input(shape=IMG_SIZE + (3,))
    x = base(inputs, training=False)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.3)(x)
    outputs = Dense(num_classes, activation='softmax',
                    dtype='float32')(x)
    return Model(inputs, outputs)

# ─── ROC plotting loop per fold ─────────────────────────────────
for fold_idx in range(1, FOLDS + 1):
    print(f"\n📊 ROC curve — Fold {fold_idx} — VGG-19")

    # split tuple (train_df, test_df) from your global `folds` list
    _, test_df = folds[fold_idx - 1]

    test_gen = val_datagen.flow_from_dataframe(
        dataframe   = test_df,
        x_col       = 'filepaths',
        y_col       = 'labels',
        target_size = IMG_SIZE,
        batch_size  = BATCH_SIZE,
        class_mode  = 'categorical',
        shuffle     = False)

    # build model & load best weights for this fold
    model = build_vgg19_model(NUM_CLASSES)
    weights_path = (f"/content/drive/MyDrive/saved_models/"
                    f"tl_vgg19_fold{fold_idx}_best.weights.h5")
    if not os.path.exists(weights_path):
        raise FileNotFoundError(f"⛔ Weights not found: {weights_path}")
    model.load_weights(weights_path)

    # predictions
    y_true      = test_gen.classes                        # integer labels
    y_true_bin  = label_binarize(y_true, classes=range(NUM_CLASSES))
    y_prob      = model.predict(test_gen, verbose=0)      # (N, 4)

    # ROC per class
    plt.figure(figsize=(8, 6))
    for i in range(NUM_CLASSES):
        fpr, tpr, _   = roc_curve(y_true_bin[:, i], y_prob[:, i])
        roc_auc       = auc(fpr, tpr)
        plt.plot(fpr, tpr, lw=2,
                 label=f"{CLASSNAMES[i]}  (AUC = {roc_auc:.2f})")

    plt.plot([0, 1], [0, 1], 'k--', lw=1)
    plt.title(f"ROC Curve – Fold {fold_idx} – VGG-19")
    plt.xlabel("False Positive Rate");  plt.ylabel("True Positive Rate")
    plt.legend(loc="lower right");  plt.grid(True);  plt.tight_layout()
    plt.show()


In [ ]:
# ================================================================
# ROC curves for the fine-tuned **DenseNet-121** on each fold
# ---------------------------------------------------------------
#  • Assumes the training cell saved:
#      /content/drive/MyDrive/saved_models/d121_fold{fold}.h5
#  • Uses the same DenseNet preprocessing
#  • Plots one-vs-rest ROC + AUC for all four classes
# ================================================================

import os, numpy as np, matplotlib.pyplot as plt, tensorflow as tf
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.densenet import DenseNet121, preprocess_input
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import mixed_precision

# ── constants ───────────────────────────────────────────────────
IMG_SIZE    = (224, 224)
BATCH_SIZE  = 48
NUM_CLASSES = 4
FOLDS       = 4
CLASSNAMES  = ["COVID", "Lung Opacity", "Normal", "Viral Pneumonia"]

# ── val generator (no augmentation) ─────────────────────────────
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# ── custom-object map for mixed precision models ────────────────
class DummyCast(tf.keras.layers.Layer):
    def call(self, inputs): return inputs

custom_objects = {"Cast": DummyCast, "Policy": mixed_precision.Policy}

# ── helper: rebuild DenseNet-121 head (needed to load weights) ──
def build_d121_model(num_classes=NUM_CLASSES):
    base = DenseNet121(include_top=False, weights="imagenet",
                       input_shape=IMG_SIZE + (3,))
    base.trainable = True           # must match training state
    inp = tf.keras.Input(shape=IMG_SIZE + (3,))
    x   = base(inp, training=False)
    x   = GlobalAveragePooling2D()(x)
    x   = Dropout(0.5)(x)
    x   = Dense(256, activation="relu")(x)
    x   = Dropout(0.3)(x)
    out = Dense(num_classes, activation="softmax", dtype="float32")(x)
    return Model(inp, out)

# ── fold loop ───────────────────────────────────────────────────
for fold_idx, (_, test_df) in enumerate(folds, start=1):
    print(f"\n📊 ROC curve — DenseNet121 — Fold {fold_idx}")

    test_gen = val_datagen.flow_from_dataframe(
        dataframe   = test_df,
        x_col       = "filepaths",
        y_col       = "labels",
        target_size = IMG_SIZE,
        batch_size  = BATCH_SIZE,
        class_mode  = "categorical",
        shuffle     = False)

    # build & load weights
    model = build_d121_model()
    weights_path = f"/content/drive/MyDrive/saved_models/mnv2_fold{fold_idx}.h5"
    with tf.keras.utils.custom_object_scope(custom_objects):
        model.load_weights(weights_path)

    # predictions
    y_true      = test_gen.classes
    y_true_bin  = label_binarize(y_true, classes=range(NUM_CLASSES))
    y_prob      = model.predict(test_gen, verbose=0)

    # plot ROC for each class
    plt.figure(figsize=(8, 6))
    for i in range(NUM_CLASSES):
        fpr, tpr, _ = roc_curve(y_true_bin[:, i], y_prob[:, i])
        roc_auc      = auc(fpr, tpr)
        plt.plot(fpr, tpr, lw=2,
                 label=f"{CLASSNAMES[i]} (AUC = {roc_auc:.2f})")

    plt.plot([0, 1], [0, 1], "k--", lw=1)
    plt.title(f"ROC Curve – DenseNet121 – Fold {fold_idx}")
    plt.xlabel("False Positive Rate");  plt.ylabel("True Positive Rate")
    plt.legend(loc="lower right");  plt.grid(True);  plt.tight_layout()
    plt.show()


In [ ]:
# ================================================================
# ROC curves from *weights-only* checkpoints of DenseNet-121
#  (files: tl_d121_fold{N}_best.weights.h5)
# ================================================================

import os, numpy as np, matplotlib.pyplot as plt, tensorflow as tf
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.densenet import DenseNet121, preprocess_input
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import mixed_precision

# ── constants ───────────────────────────────────────────────────
IMG_SIZE    = (224, 224)
BATCH_SIZE  = 48
NUM_CLASSES = 4
FOLDS       = 4
CLASSNAMES  = ["COVID", "Lung Opacity", "Normal", "Viral Pneumonia"]

# ── val generator (no augmentation) ─────────────────────────────
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# ── build DenseNet-121 with the *same* head you trained ----------
def build_d121_model(num_classes=NUM_CLASSES):
    base = DenseNet121(include_top=False, weights="imagenet",
                       input_shape=IMG_SIZE + (3,))
    inp = tf.keras.Input(shape=IMG_SIZE + (3,))
    x   = base(inp, training=False)
    x   = GlobalAveragePooling2D()(x)
    x   = Dropout(0.5)(x)
    x   = Dense(256, activation="relu")(x)
    x   = Dropout(0.3)(x)
    out = Dense(num_classes, activation="softmax", dtype="float32")(x)
    return Model(inp, out)

# ── custom-object map for mixed precision (if model saved w/ MP) –
class DummyCast(tf.keras.layers.Layer):
    def call(self, inputs): return inputs
custom_objects = {"Cast": DummyCast, "Policy": mixed_precision.Policy}

# ── fold loop ───────────────────────────────────────────────────
for fold_idx, (_, test_df) in enumerate(folds, start=1):
    print(f"\n📊 ROC curve — DenseNet121 — Fold {fold_idx}")

    test_gen = val_datagen.flow_from_dataframe(
        dataframe   = test_df,
        x_col       = "filepaths",
        y_col       = "labels",
        target_size = IMG_SIZE,
        batch_size  = BATCH_SIZE,
        class_mode  = "categorical",
        shuffle     = False)

    # build fresh model & load weights-only checkpoint
    model = build_d121_model()
    w_path = (f"/content/drive/MyDrive/saved_models/"
              f"tl_d121_fold{fold_idx}_best.weights.h5")
    if not os.path.exists(w_path):
        raise FileNotFoundError(w_path)
    model.load_weights(w_path)

    # predictions
    y_true      = test_gen.classes
    y_true_bin  = label_binarize(y_true, classes=range(NUM_CLASSES))
    y_prob      = model.predict(test_gen, verbose=0)

    # plot ROC for each class
    plt.figure(figsize=(8, 6))
    for i in range(NUM_CLASSES):
        fpr, tpr, _ = roc_curve(y_true_bin[:, i], y_prob[:, i])
        roc_auc      = auc(fpr, tpr)
        plt.plot(fpr, tpr, lw=2,
                 label=f"{CLASSNAMES[i]} (AUC = {roc_auc:.2f})")

    plt.plot([0, 1], [0, 1], "k--", lw=1)
    plt.title(f"ROC Curve – DenseNet121 – Fold {fold_idx}")
    plt.xlabel("False Positive Rate");  plt.ylabel("True Positive Rate")
    plt.legend(loc="lower right");  plt.grid(True);  plt.tight_layout()
    plt.show()
